# Tâche #3 : Classification d'incidents avec des modèles *Transformers*

Toujours avec la même tâche et les mêmes fichiers de textes, utiliser la librairie HuggingFace pour accomplir cette tâche. On demande plus spécifiquement d’utiliser le modèle bert-base-uncased et un autre modèle de votre choix. 
Les consignes associées à cette tâche sont: 
-	Nom du notebook : transformer.ipynb
-	Tokenisation : Celle fournie par les tokeniseurs accompagnant les modèles transformers. 
-	Plongements de mots : Ceux du modèle transformer. 
-	Normalisation : Lettre en minuscule pour Bert. Aucune contrainte pour le 2e modèle. 
-	Choix du 2e transformer: Un modèle encodeur préentraîné pour l’anglais. Le modèle ne doit pas être une autre version de Bert et doit être significativement différent. Utilisez un 2 fichier pour ce modèle si nécessaire (une copie de celui-ci). 
-	Analyse : Comparer les résultats obtenus avec les 2 modèles transformers. Présentez également une comparaison globale des résultats obtenus avec tous les modèles utilisés dans ce travail et ceux du travail précédent (TP #1). 


Vous pouvez ajouter au *notebook* toutes les cellules dont vous avez besoin pour votre code, vos explications ou la présentation de vos résultats. Vous pouvez également ajouter des sous-sections (par ex. des sous-sections 1.1, 1.2 etc.) si cela améliore la lisibilité.

Notes :
- Évitez les bouts de code trop longs ou trop complexe. Par exemple, il est difficile de comprendre 4-5 boucles ou conditions imbriquées. Si c'est le cas, définissez des sous-fonctions pour refactoriser et simplifier votre code. 
- Expliquez sommairement votre démarche.
- Expliquez les choix que vous faites au niveau de la programmation et des modèles (si non trivial).
- Analyser vos résultats. Indiquez ce que vous observez, si c'est bon ou non, si c'est surprenant, etc. 
- Une analyse quantitative et qualitative d'erreurs est intéressante et permet de mieux comprendre le comportement d'un modèle.

## 1. Création du jeu de données (*dataset*)

In [39]:
# Importation des bibliothèques nécessaires
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import classification_report

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [41]:
class IncidentDataset(Dataset):
    def __init__(self, file, tokenizer, max_length):
        with open(file, 'r') as f:
            self.data = json.load(f)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx]['text']
        label = int(self.data[idx]['label'])
        # Tokenisation avec le tokeniseur du modèle
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        inputs = {key: val.squeeze() for key, val in inputs.items()}
        inputs['labels'] = torch.tensor(label)  # Add the label to the dictionary
        return inputs  # Now we return a dictionary

## 2. Création de modèle(s)

In [42]:
# Initialiser les tokeniseurs et les modèles
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=9).to(device)

roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=9).to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
model=bert_model

## 3. Entraînement de modèle(s)

In [44]:
# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Initialiser le DataLoader
train_data = IncidentDataset('data/incidents_train.json', bert_tokenizer, 512)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

# Boucle d'entraînement
for epoch in range(1):  # 10 = nombre d'époques
    for i, inputs in enumerate(train_loader):
        inputs = {name: tensor.to(device) for name, tensor in inputs.items()}  # Move input tensors to the device
        labels = inputs.pop('labels')  # Remove labels from inputs
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

: 

## 4. Évaluation et analyse de résultats

In [ ]:
# Évaluation du modèle sur les données de test
model.eval()
predictions, true_labels = [], []
for inputs in test_loader:
    with torch.no_grad():
        outputs = model(**inputs)
    _, predicted = torch.max(outputs.logits, 1)
    predictions.extend(predicted.tolist())
    true_labels.extend(inputs['labels'].tolist())

# Afficher le rapport de classification
print(classification_report(true_labels, predictions))

NameError: name 'test_loader' is not defined

In [1]:
import pandas as pd

# User Data
users_data = {
    'User_ID': [1, 2, 3],
    'Age': [28, 35, 30],
    'Income': [60000, 80000, 70000],
    'Debt': [5000, 2000, 0],
    'Savings': [2000, 5000, 3000],
    'Gender': ['Male', 'Female', 'Male']
}

users_df = pd.DataFrame(users_data)


In [8]:


# Financial Advice Data
advice_data = {
    'Advice_ID': [501, 502, 503, 504],
    'Title': ['Create Emergency Fund', 'Budgeting Tips', 'Invest in Retirement', 'Pay Off High-Interest Debt'],
    'Description': [
        'Save 3-6 months\' worth of living expenses.',
        'Track your spending and create a monthly budget.',
        'Contribute to a retirement savings account.',
        'Focus on paying down debts with high interest.'
    ],
    'Category': ['Savings', 'Budgeting', 'Investments', 'Debt Management']
}

advice_df = pd.DataFrame(advice_data)



In [9]:

# User-Advice Interactions Data
interactions_data = {
    'User_ID': [1, 1, 2, 3],
    'Advice_ID': [501, 502, 503, 504],
    'Rating': [4, 5, 4, 3],
    'Timestamp': ['2023-01-19 09:00:00', '2023-01-20 14:30:00', '2023-01-22 11:45:00', '2023-01-23 08:20:00']
}

interactions_df = pd.DataFrame(interactions_data)



In [10]:
users_df

,User_ID,Age,Income,Debt,Savings,Gender
0,1,28,60000,5000,2000,Male
1,2,35,80000,2000,5000,Female
2,3,30,70000,0,3000,Male


In [11]:
advice_df

,Advice_ID,Title,Description,Category
0,501,Create Emergency Fund,Save 3-6 months' worth of living expenses.,Savings
1,502,Budgeting Tips,Track your spending and create a monthly budget.,Budgeting
2,503,Invest in Retirement,Contribute to a retirement savings account.,Investments
3,504,Pay Off High-Interest Debt,Focus on paying down debts with high interest.,Debt Management


In [12]:
interactions_df

,User_ID,Advice_ID,Rating,Timestamp
0,1,501,4,2023-01-19 09:00:00
1,1,502,5,2023-01-20 14:30:00
2,2,503,4,2023-01-22 11:45:00
3,3,504,3,2023-01-23 08:20:00
